# Reinforcement Learning Notes: Chapter 2

## K-Armed Bandit Problem

We define the (stationary) k-armed bandit problem as follows: We repeatedly face a choice of k actions. There is an associated numerical reward chosen from a (stationary) probability that depends on the action chosen. The goal is to maximize the expected total reward over some _time steps_.

## Action-Value Estimation Methods

We first discuss a solution to the k-Armed Bandit problem using the so-called Action-Value Estimation methods. The idea is to come up with an estimate of the _expected reward_ of an action, also called the action's _value_. Suppose we knew the values of each action, then it would be trivial to solve the k-armed bandit problem: we just always select the arm with the highest value.

A straightforward to estimate the action-values is to do random exploration and use the some kind of average of the rewards received for each action as its estimated value.


Terminologies:
* $Q_{n}(a): = $ estimated value of action $a$ after selecting it $n-1$ times. In another words, it is the estimated value of $a$ before selecting it for the $n$th time. We sometimes use $Q_{t}(a)$ to denote the estimated value of $a$ _prior_ to time step $t$.
* $R_{n}(a): = $ reward received after the selection of action $a$ for the $n$th time. Similarly for $R_t(a)$.

### Simple Average Estimate

In this method, we estimate $Q_{t}(a)$ by the simple (unweighted) average of all previously observed rewards received after taking action $a$ _prior to time step t_:
$$
Q_{t}(a) := \frac{\sum_{i = 1}^{t-1}\chi_{A_i = a}\cdot R_i(a)}{\sum_{i = 1}^{t-1}\chi_{A_i = a}},
$$
where the characteristic function $\chi_{A_i = a}$ takes the value of $1$ if the $i$th action chosen is $a$, and $0$ otherwise. We then select the action with the maximum estimated value at time $t$:
$$
A_t := argmax_{a} Q_t(a).
$$

From an algorithmic point of view, for each action we keep track of the number of times it is selected as well as the historical rewards (incrementally) to push the estimated value forward in time. This is where the $Q_n$ notation becomes handy: it can be easily shown that:

\begin{equation}
Q_{n+1} = Q_{n} + \frac{1}{n}\left[R_{n} - Q_{n} \right]
\end{equation}

This incremental update form is going to occur frequently in the book. We can think of it as:

$$
\boxed{New Estimate := Old Estimate + StepSize * [Target - Old Estimate]}.
$$
Here we take the step size to be $\frac{1}{n}$, for an action that has been chosen $n$ times before. We can have the following pesudo-code for the simple average estimation method:

In [ ]:
# Pseudo-code for the sample average multi-armed bandit
def multi_armed_bandit:
  # Initialize Q and N for each action to be 0
  for a in range(k):
    Q[a] = 0
    N[a] = 0
  for epoch in range(MAX_EPOCHS):
    # pull arm function that selects optimal action given action-value array
    A = pull_arm(Q)
    cur_reward = bandit(A)

    # update the MAB system
    N[a] = N[a] + 1
    Q[a] = Q[a] + (cur_reward - Q[a]) / N[a]

### Non-stationary Problems: Exponential Recency Weighted Average

The simple average estimate makes sense in the case where the reward probabilities are stationary (i.e. they do not change over time). In the case of a non-stationary k-armed bandit problem, it makes more sense to have a weighted average where we give more weights on the more recent reward in our estimate. One way to achieve this is to use a **constant** step size in the update rule:

$$
Q_{n+1} = Q_{n} + \alpha[R_{n} - Q_{n}], \quad \alpha \in (0, 1].
$$

It can be shown that:
    
$$
Q_{n+1} = (1 - \alpha)^n Q_{1} + \sum_{i = 1}^{n}\alpha (1 - \alpha)^{n-i}R_{i}.
$$

The weight $\alpha (1 - \alpha)^{n-i}$ is given to the reward $R_{i}$, which is observed $n-i$ rewards ago. Since $\alpha \in (0, 1]$, the weight of $R_{i}$ decreases _exponentially_ as the number of interleaving rewards increases. Therefore, we also call this method _Exponential Recency weighted Average_.

If we let $\alpha_{n}(a)$ to denote the step size with respect to updating the action-value of $a$ at step $n$. Then the simple average estimate corresponds to $\alpha_{n}(a) = \frac{1}{n}$, and the exponential recency weighted average estimate corresponds to $\alpha$. 



### Upper-Confidence-Bound Action Selection

In $\epsilon$-greedy algorithm, we are forced to do exploration $\epsilon$ percent of the time and in a completely random fashion. It would be better to have some intelligence in the exploration so that we favor actions that have a higher potential of higher reward. One effective way of doing that is to select the action according to:

$$
A_{t} := argmax_{a} \left[Q_{t}(a) + c\sqrt{\frac{\ln{t}}{N_{t}(a)}} \right].
$$

Here $Q_{t}$ is the estimated reward prior to time step $t$. $N_{t}(a)$ is the number of times action $a$ has been chosen priot to time step $t$. $c$ is a hyper-parameter that controls the exploration. If $N_{t}(a) = 0$, then $a$ is considered to be a maximizing action. This method is called the *Upper-Confidence-Bound* action selection.

The square root term represents the uncertainty or variance in the estimate of the value of $a$. The whole term represents the _upper bound_ on the possible true value of $a$ with $c$ controlling the level of confidence. The more times we have seen $a$ the less uncertainty there is.

## Gradient Bandit Algorithm

So far we have considered methods that estimate action values and use those estimates to select actions. We can also consider learning a numerical _preference_ for each action $a$, denoted as $H_t(a)$. The larger the preference, the more often that action is taken, but the preference has no interpretation in terms of reward. Given such a preference function we can select the action according to the **soft-max distribution**:

$$
\pi_{t}(a) := \mathbb{P}\{{A_t = a}\} = \frac{e^{H_t(a)}}{\sum_{b = 1}^{k}e^{H_t(b)}}.
$$

Here $\pi_{t}(a)$ denotes the probability that action $a$ is selected at time step $t$. We initialize the preference functions to be the same across all actions (e.g. $H_1(a) = 0, \forall a$). Note that in the case of a 2-armed bandit problem, the distribution becomes a logistic function.

### Update Rule

At each step after selecting action $A_t$ and receiving reward $R_t$ we update the action preferences by:
    
\begin{align}
H_{t+1}(A_t) & := H_t(A_t) + \alpha(R_t - \bar{R_t})(1 - \pi_t(A_t)), \quad \text{and} \\
H_{t+1}(a) & := H_t(a) - \alpha(R_t - \bar{R_t})\pi_t(a), \quad\quad\quad\quad \text{for all $a \neq A_t$}
\end{align}

where $\alpha > 0$ is a step-size parameter, and $\bar{R_t}$ is the average of all the rewards up through and including time step $t$ which can be computed incrementally. The $\bar{R_t}$ terms serves as a baseline against which the current reward is compared. If the current reward is higher than the baseline, then the preference of taking $A_t$ in the future is increased, and if the reward is below the baseline, then the reference id decreased. The non-selected actions move in the opposite direction.

### Stochastic Gradient Descent

In a pure SGD framework, we would want to move $H_t$ in gradient descent direction of minimizing our objection function, which is the expected reward:

$$
H_{t+1}(a) : = H_t(a) + \alpha \nabla_{H_t(a)}\mathbb{E}[R_t],
$$

where 
$$
\mathbb{E}[R_t] = \sum_{x} \pi_t(x) q_*(x).
$$

It can be shown that:
$$
\nabla_{H_t(a)}\mathbb{E}[R_t] = \mathbb{E}[(R_t - \bar{R_t})(\chi_{a = A_t} - \pi_t(a))], \quad \forall a.
$$

Therefore the update rule above is an instance of the gradient update rule, justifying the term _gradient bandit algorithm_.